In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense,Flatten,Dropout,Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50, resnet50, Xception, xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.activations import softmax
from sklearn.model_selection import train_test_split

import cv2
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [3]:
NUM_CLASSES = 5
CHANNELS = 3
IMAGE_RESIZE = 224
NUM_EPOCHS = 100
EARLY_STOP_PATIENCE = 10

BATCH_SIZE_TRAINING = 32
BATCH_SIZE_VALIDATION = 32
# Using 1 to easily manage mapping between test_generator & prediction for submission preparation
BATCH_SIZE_TESTING = 1

# Transfer learning with Xception 

In [4]:
model = Sequential()
model.add(Xception(include_top=False, weights='imagenet', input_shape=(IMAGE_RESIZE, IMAGE_RESIZE, CHANNELS), pooling='avg'))
model.add(Dense(NUM_CLASSES, activation=softmax))
model.layers[0].trainable = False
model.summary()

83689472/83683744 [==============================] - 4s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 2048)              20861480  
_________________________________________________________________
dense (Dense)                (None, 5)                 10245     
Total params: 20,871,725
Trainable params: 10,245
Non-trainable params: 20,861,480
_________________________________________________________________


In [5]:
model.compile(loss=CategoricalCrossentropy(), 
              optimizer=tf.keras.optimizers.SGD(0.1, momentum=0.9, nesterov=True, decay=0.01), #decay=0.01
              #optimizer=tf.keras.optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, amsgrad=False),
              metrics=['accuracy'])

In [6]:
callbacks = [
    EarlyStopping(monitor='val_accuracy', mode='max', restore_best_weights=True, patience=EARLY_STOP_PATIENCE),
    ModelCheckpoint(filepath='../working/flowers_transfer_model.h5', monitor='val_accuracy', mode='max', save_best_only=True)
]

In [7]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
    zoom_range = 0.1, # Randomly zoom image 
    width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    validation_split=0.2
)

In [8]:
train_data_gen = data_generator.flow_from_directory(
    '../input/flowers-recognition/flowers/flowers/',
    target_size=(IMAGE_RESIZE,IMAGE_RESIZE),
    batch_size=BATCH_SIZE_TRAINING,
    class_mode='categorical',
    shuffle=True,
    seed=0,
    subset="training" #constant string
)

valid_data_gen = data_generator.flow_from_directory(
    '../input/flowers-recognition/flowers/flowers/',
    target_size=(IMAGE_RESIZE,IMAGE_RESIZE),
    batch_size=BATCH_SIZE_VALIDATION,
    class_mode='categorical',
    shuffle=True,
    seed=0,
    subset="validation" #constant string
)

Found 3462 images belonging to 5 classes.
Found 861 images belonging to 5 classes.


In [9]:
history = model.fit_generator(train_data_gen,
                              steps_per_epoch=train_data_gen.samples//BATCH_SIZE_TRAINING,
                              validation_data=valid_data_gen,
                              validation_steps=valid_data_gen.samples//BATCH_SIZE_VALIDATION,
                              epochs=NUM_EPOCHS,
                              callbacks = callbacks
                             )

Train for 108 steps, validate for 26 steps
Epoch 1/100
108/108 [==============================] - 75s 697ms/step - loss: 0.6745 - accuracy: 0.7519 - val_loss: 0.8626 - val_accuracy: 0.7452
Epoch 2/100
108/108 [==============================] - 61s 561ms/step - loss: 0.4514 - accuracy: 0.8434 - val_loss: 0.9098 - val_accuracy: 0.7608
Epoch 3/100
108/108 [==============================] - 59s 548ms/step - loss: 0.4407 - accuracy: 0.8397 - val_loss: 0.8311 - val_accuracy: 0.7476
Epoch 4/100
108/108 [==============================] - 59s 543ms/step - loss: 0.3798 - accuracy: 0.8679 - val_loss: 0.7431 - val_accuracy: 0.7788
Epoch 5/100
108/108 [==============================] - 58s 535ms/step - loss: 0.3727 - accuracy: 0.8685 - val_loss: 0.8840 - val_accuracy: 0.7644
Epoch 6/100
108/108 [==============================] - 58s 533ms/step - loss: 0.3633 - accuracy: 0.8741 - val_loss: 0.8760 - val_accuracy: 0.7464
Epoch 7/100
108/108 [==============================] - 57s 529ms/step - loss: 0.3

# Retrain the whole network with smaller learning rate

In [10]:
model.layers[0].trainable = True

In [11]:
model.compile(loss=CategoricalCrossentropy(),  
              optimizer=tf.keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=0.001, nesterov=True),
              #optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False),
              metrics=['accuracy'])

In [12]:
history = model.fit_generator(train_data_gen,
                              steps_per_epoch=train_data_gen.samples//BATCH_SIZE_TRAINING,
                              validation_data=valid_data_gen,
                              validation_steps=valid_data_gen.samples//BATCH_SIZE_VALIDATION,
                              epochs=NUM_EPOCHS,
                              callbacks = callbacks
                             )

Train for 108 steps, validate for 26 steps
Epoch 1/100
108/108 [==============================] - 78s 721ms/step - loss: 0.3080 - accuracy: 0.8913 - val_loss: 0.5182 - val_accuracy: 0.8486
Epoch 2/100
108/108 [==============================] - 72s 663ms/step - loss: 0.1092 - accuracy: 0.9653 - val_loss: 0.3739 - val_accuracy: 0.8954
Epoch 3/100
108/108 [==============================] - 72s 665ms/step - loss: 0.0449 - accuracy: 0.9857 - val_loss: 0.3708 - val_accuracy: 0.9111
Epoch 4/100
108/108 [==============================] - 72s 665ms/step - loss: 0.0272 - accuracy: 0.9918 - val_loss: 0.3173 - val_accuracy: 0.9243
Epoch 5/100
108/108 [==============================] - 71s 654ms/step - loss: 0.0172 - accuracy: 0.9959 - val_loss: 0.3316 - val_accuracy: 0.9147
Epoch 6/100
108/108 [==============================] - 71s 660ms/step - loss: 0.0085 - accuracy: 0.9980 - val_loss: 0.3070 - val_accuracy: 0.9231
Epoch 7/100
108/108 [==============================] - 72s 668ms/step - loss: 0.0

### Connect/Save at jovian.ml 

In [13]:
!pip install jovian --upgrade --quiet

In [14]:
import jovian

<IPython.core.display.Javascript object>

In [ ]:
jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..


In [ ]:
%capture [--no-stderr] [--no-stdout] [--no-display] [output]